![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [39]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.impute import SimpleImputer

# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data",header=None) 
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+


In [40]:
cc_apps.describe()

,2,7,10,12
count,690.000000,690.000000,690.00000,690.000000
mean,4.758725,2.223406,2.40000,1017.385507
std,4.978163,3.346513,4.86294,5210.102598
min,0.000000,0.000000,0.00000,0.000000
25%,1.000000,0.165000,0.00000,0.000000
50%,2.750000,1.000000,0.00000,5.000000
75%,7.207500,2.625000,3.00000,395.500000
max,28.000000,28.500000,67.00000,100000.000000


In [41]:
cc_apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    object 
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    int64  
 13  13      690 non-null    object 
dtypes: float64(2), int64(2), object(10)
memory usage: 75.6+ KB


In [42]:
# Preprocessing the data
cc_apps_new = cc_apps.replace('?', np.nan)
cc_apps_new.head()
print(cc_apps_new.isna().sum())
print(cc_apps_new.info())

0     12
1     12
2      0
3      6
4      6
5      9
6      9
7      0
8      0
9      0
10     0
11     0
12     0
13     0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       678 non-null    object 
 1   1       678 non-null    object 
 2   2       690 non-null    float64
 3   3       684 non-null    object 
 4   4       684 non-null    object 
 5   5       681 non-null    object 
 6   6       681 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    int64  
 13  13      690 non-null    object 
dtypes: float64(2), int64(2), object(10)
memory usage: 75.6+ KB
None


In [43]:
cc_apps_impute = cc_apps_new.copy()

for col in cc_apps_impute.columns:
    if cc_apps_impute[col].dtypes == 'object':
        imp_cat = SimpleImputer(strategy="most_frequent")
        cc_apps_impute[col] = imp_cat.fit_transform(cc_apps_impute[[col]])
    else:
        imp_num = SimpleImputer()
        cc_apps_impute[col] = imp_num.fit_transform(cc_apps_impute[[col]])

In [44]:
cc_apps_impute.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    object 
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    float64
 11  11      690 non-null    object 
 12  12      690 non-null    float64
 13  13      690 non-null    object 
dtypes: float64(4), object(10)
memory usage: 75.6+ KB


In [45]:
# dummify the categorical features
cc_apps_encoded = pd.get_dummies(cc_apps_impute, drop_first = True)

In [46]:
#Split the dataframe into features and target variable
X = cc_apps_encoded.iloc[:, :-1].values
y = cc_apps_encoded.iloc[:, [-1]].values

In [47]:
# split the values into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)


In [48]:
# scale the values
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [49]:
#Train the model
logreg = LogisticRegression()
logreg.fit(X_train_scaled, y_train)

#pred from the train data
y_train_pred = logreg.predict(X_train_scaled)

In [50]:
#print the confusion matrix
print(confusion_matrix(y_train, y_train_pred))

[[222   3]
 [  3 289]]


In [51]:
print(y_train.shape)

(517, 1)


In [52]:
y_test_pred = logreg.predict(X_test_scaled)
print(confusion_matrix(y_test, y_test_pred))

[[61 21]
 [18 73]]


In [57]:
# hyperparameter tuning
tol = [0.01, 0.001, 0.0001, 0.00001]
max_iter = [100, 150,200, 250]

param_grid = dict(tol=tol, max_iter=max_iter)
# kf = KFold(n_splits= 5, shuffle = True, random_state = 42)
grid_model = GridSearchCV(logreg, param_grid = param_grid, cv = 5)


In [58]:
#train with new model
grid_model_fit = grid_model.fit(X_train_scaled, y_train)
grid_model_train_pred = grid_model.predict(X_test_scaled)
print(confusion_matrix(y_test, grid_model_train_pred))

[[61 21]
 [18 73]]


In [59]:
# Summarize results
best_train_score, best_train_params = grid_model_fit.best_score_, grid_model_fit.best_params_
print("Best: %f using %s" % (best_train_score, best_train_params))

# Extract the best model and evaluate it on the test set
best_model = grid_model_fit.best_estimator_
best_score =  best_model.score(X_test_scaled, y_test)

print("Accuracy of logistic regression classifier: ", best_score)

Best: 0.814339 using {'max_iter': 100, 'tol': 0.01}
Accuracy of logistic regression classifier:  0.7745664739884393
